In [2]:
import pandas as pd
print('Pandas version is %s' % pd.__version__)
import numpy as np
print('NumPy version is %s' % np.__version__)
import statsmodels.api as sm
from statsmodels.robust.scale import mad as mad_function
print('Statsmodels version is %s' % sm.__version__)
import scipy.stats as stats

import matplotlib
import matplotlib.pyplot as plt
print('Matplotlib version is %s' % matplotlib.__version__)
import seaborn as sns
print('Seaborn version is %s' % sns.__version__)
%matplotlib inline
%pylab inline

import time
import random
import itertools
print('Imports done.')


Pandas version is 1.2.3
NumPy version is 1.20.1
Statsmodels version is 0.13.2
Matplotlib version is 3.3.4
Seaborn version is 0.11.1
Populating the interactive namespace from numpy and matplotlib
Imports done.


# Reading in and cleaning up the data

In [53]:
### Read in data ###
data_mr_young = pd.read_excel('miniTRK_APS_Recognition_Trial-by-Trial_Data.xlsx')
indeces = data_mr_young[data_mr_young['Correct'] == -1].index  # -1 in 'Correct' column denotes missing response
data_mr_young = data_mr_young.drop(indeces, axis=0)

data_online = pd.read_excel('longTRK_APS_Recognition_Trial-by-Trial_Data.xlsx')
data_online.rename(columns={'Online ID':'online ID'}, inplace=True) # Renaming for consistency

# Splitting young and old online data
data_online_young = data_online[data_online['Group']==1]
data_online_old = data_online[data_online['Group']==0]
data_online_old = data_online_old.replace("410O824F)", "410O824F")
data_list = [data_mr_young, data_online_young, data_online_old, data_online]
data_names = ['data_mr_young', 'data_online_young', 'data_online_old', 'data_online']
display(data_online.head(), data_mr_young.head())


,online ID,Group,TrialIndex,TrialType,StimType,EncodingStimType,rec_trial_key.keys,rec_trial_key.rt,CurrentImage,EncodingImage,...,TripletMemberB,TripletMemberC,CurrentX,CurrentY,Xcoordinate,Ycoordinate,Xcoordinate_lure1,Ycoordinate_lure1,Correct,ResponseNew
0,908O1823F,0,1,OBJ,FOIL,FOIL,j,2.7586,stimuli/fractals/59c.jpg,NaN,...,NaN,NaN,0.304695,0.379784,NaN,NaN,NaN,NaN,1,1
1,908O1823F,0,2,OBJ,LURE,ERP,f,1.4135,stimuli/fractals/163c.jpg,stimuli/fractals/163b.jpg,...,stimuli/fractals/163a.jpg,stimuli/fractals/163c.jpg,0.376897,0.051308,0.376897,0.051308,0.182213,-0.364402,0,0
2,908O1823F,0,3,OBJ,TARGET,ERP,j,2.3935,stimuli/fractals/119b.jpg,stimuli/fractals/119b.jpg,...,stimuli/fractals/119c.jpg,stimuli/fractals/119a.jpg,-0.205913,0.156378,-0.205913,0.156378,-0.361948,-0.338631,0,1
3,908O1823F,0,4,OBJ,LURE,ERP,f,1.7771,stimuli/fractals/147a.jpg,stimuli/fractals/147b.jpg,...,stimuli/fractals/147c.jpg,stimuli/fractals/147a.jpg,-0.380412,-0.034671,-0.380412,-0.034671,-0.125804,-0.194251,0,0
4,908O1823F,0,5,OBJ,FOIL,FOIL,f,3.4714,stimuli/fractals/161c.jpg,NaN,...,NaN,NaN,0.109361,0.393758,NaN,NaN,NaN,NaN,0,0


,online ID,MR ID,Stimuli Table,TrialIndex,TrialType,StimType,EncodingStimType,rec_trial_key.keys,rec_trial_key.rt,CurrentImage,...,TRUE_RGB2_BIN,TRUE_RATING2_BIN,DIST1,DIST1_BIN,DIST2,DIST2_BIN,TRUE_DIST2,TRUE_DIST2_BIN,Correct,ResponseNew
0,101mTRK21M,275592,2,1,OBJ,FOIL,FOIL,c,1.356678,stimuli/fractals/153a.jpg,...,foil,foil,3.026812e+02,3,302.681192,3,3.026812e+02,3,1,1
1,101mTRK21M,275592,2,2,OBJ,LURE,ERP,b,1.532706,stimuli/fractals/25b.jpg,...,medium,high,0.000000e+00,0,292.101183,2,0.000000e+00,0,0,0
2,101mTRK21M,275592,2,3,OBJ,TARGET,ERP,c,1.720454,stimuli/fractals/62c.jpg,...,target,target,2.842171e-14,0,306.414435,3,2.842171e-14,0,0,1
3,101mTRK21M,275592,2,4,OBJ,LURE,ERP,b,1.995551,stimuli/fractals/73b.jpg,...,high,medium,5.684342e-14,0,259.168591,2,5.684342e-14,0,0,0
4,101mTRK21M,275592,2,5,OBJ,FOIL,FOIL,c,1.295759,stimuli/fractals/108c.jpg,...,foil,foil,4.308871e+02,4,430.887072,4,4.308871e+02,4,1,1


In [ ]:
### Functions for preprocessing ###

def filter_completed(data, criteria=None, id_var='online ID'):
    """
    Filter out participants having less completed trials than criterion. Print number of deleted participants.

    Parameters
    ----------
    data: dataframe
    criteria: float or None
        Minimum number of trials for a participant to be included
    id_var: str
        Name of column containing participant names

    Returns
    -------
    dataframe
        Filtered data

    """
    participant_completed = [name for name in data[id_var].unique() if (data[data[id_var]==name][id_var].value_counts() >= criteria).all()]
    difference = len(data[id_var].unique()) - len(participant_completed)
    print('Deleted participants:', difference)

    data = data[data[id_var].isin(participant_completed)]

    return data


In [ ]:
### Data preparation ###

# Filter out participants with more than 10% missing trials
for i in range(len(data_list)):
    data = data_list[i]
    data_list[i] = filter_completed(data, 144*0.9)

# Combined dataframe for plotting
for i in range(len(data_list)):
    df = data_list[i]
    df.loc[:,'data_set'] = data_names[i]
    data_list[i] = df
data_all = pd.concat([data_list[i] for i in range(len(data_list))])


# Calculating effect sizes

In [5]:
def edge_correction(proportion, all):
    """
    Perform edge-correction to avoid infinite values in d-prime calculations.

    Parameters
    ----------
    proportion : float
        Quantity to check.
    all: float
        Quantity to use in correction, e.g. hits + misses, false alarms + correct rejections, etc.

    Returns
    -------
    float
    
    """
    # Should we exclude participants instead?
    if proportion >= 1:
        proportion = 1 - (0.5/all)
    elif proportion <= 0:
        proportion = 0.5/all
    return proportion

def discrimination_calculator(data, id_var='online ID', stimtype_var='StimType', response_var='ResponseNew'):
    """
    Compute d-prime as a measure of discrimination in the APS task. D-prime is calculated as the difference in 
    the z-transformed proportions of 'old' answers to 'target' stimuli and 'old' answers to 'lure' stimuli.

    Parameters
    ----------
    data : dataframe
        Long format dataframe with a column for subject, stimulus type and response (new/old).
    id_var : str
        Name of column containing participant names.
    stimtype_var : str
        Name of column containing stimulus type in trial.
    response_var : str
        Name of column containing responses. 'New' should be coded as 1, 'old' should be coded as 0.
    filters : list of str
        List of regressors to include in d-prime calculation for noise filtering.

    Returns
    -------
    dataframe
        Dataframe containing a column for participant ID and a column for their respective d-prime values.
    
    """

    # TODO Is edge-correction the best solution to deal with infs?
    # Edge-correction: https://lindeloev.net/calculating-d-in-python-and-php/
    # adjusted ratios: https://github.com/neuropsychology/psycho.R/blob/master/R/dprime.R
    # or just returning nan.

    discrimination_df = pd.DataFrame(columns=[id_var, 'd_prime', 'p_old_target', 'p_old_lure']) # Setup output dataframe
    for id in data[id_var].unique():
        # Looping through each participant's data
        data_participant = data[data[id_var]==id]
        all_target = len(data_participant[data_participant[stimtype_var]=='TARGET'])
        all_lure = len(data_participant[data_participant[stimtype_var]=='LURE'])

        # Calculate proportion of old-target answers to all answers and old-lure answers to all answers
        p_old_target = len(data_participant[(data_participant[response_var]==0) & (data_participant[stimtype_var]=='TARGET')]) / all_target
        p_old_lure = len(data_participant[(data_participant[response_var]==0) & (data_participant[stimtype_var]=='LURE')]) / all_lure
        
        # Avoiding infinite values by edge-correction
        p_old_target = edge_correction(p_old_target, all_target)
        p_old_lure = edge_correction(p_old_lure, all_lure)
                
        discrimination = stats.norm.ppf(p_old_target) - stats.norm.ppf(p_old_lure)
        discrimination_df = discrimination_df.append({id_var:id, 'd_prime':discrimination, 'p_old_target':p_old_target,'p_old_lure':p_old_lure}, ignore_index=True)
    
    return discrimination_df


# Even-odd split-half reliability with Spearman-Brown correction

In [28]:
def correl_ci(coefficient, CI, data):
    """
    Calculate confidence interval of correlation coefficient using Fisher's Z-transform.
    
    Paramters
    ----------
    coefficient: float
        The correlation coefficient.
    CI: {90, 95, 99}
        The desired confidence level.
    data: dataframe
        The dataframe from which the correlation coefficient was calculated.

    Returns
    -------
    list
        First element is the correlation coefficient, second element is the lower bound
        of the CI and the third element is the upper bound of the CI.
    
    """

    #CI transform
    if CI == 90:
        z_CI = 1.645
    elif CI == 95:
        z_CI = 1.96
    elif CI == 99:
        z_CI = 2.576
    
    #Calculate
    datan = len(data)
    z_r = 0.5 * np.log((1+coefficient) / (1-coefficient))
    z_low = z_r - z_CI * np.sqrt(1.0/(datan-3))
    z_high = z_r + z_CI * np.sqrt(1.0/(datan-3))
    
    #Convert back
    r_low = ((np.exp(2 * z_low)) - 1) / ((np.exp(2 * z_low)) + 1)
    r_high = ((np.exp(2 * z_high)) - 1) / ((np.exp(2 * z_high)) + 1)

    return [coefficient, r_low, r_high]

def split_var(length):
    """
    Make a list of given length of alternating 0s and 1s.

    Parameters
    ----------
    length: int
        Length of list
    
    Returns
    -------
    list

    """
    a = np.empty((length,))
    a[::2] = 1
    a[1::2] = 0
    return a.tolist()

def assign_even_odd(data, id_var):
    """
    Split data into even and odd trials for each participant.

    Parameters
    ----------
    data: dataframe
    id_var: string
        Name of the column holding participant names
    
    Returns
    -------
    dataframe
        The data containing even numbered trials
    dataframe
        The data containing odd numbered trials
    dataframe
        The original data
    """
    data_sorted = pd.DataFrame()
    for participant in data[id_var].unique():
        data_participant = data[data[id_var] == participant].sort_values('StimType')  # Stratified even-odd (Pronk, 2021)
        splitting_list = split_var(len(data_participant))
        data_participant.loc[:,'split_var'] = splitting_list
        data_sorted = pd.concat([data_sorted, data_participant])

    data_even = data_sorted[data_sorted['split_var']==0]
    data_odd = data_sorted[data_sorted['split_var']==1]

    return data_even, data_odd, data

def splithalf(data, id_var='online ID', nonparametric=False):
    """
    Calculate split-half reliability with and without Spearman-Brown correction.

    Parameters
    ----------
    data: dataframe
    id_var: str
        Name of the column holding participant names
    nonparametric: bool
        Calculate Spearman's rank-order correlation if True, Pearson's if False
    method: {'regression', 'difference}
        See discrimination_calculator for details.
    regressors: list of str
        See discrimination_calculator for details.

    Returns
    -------
    list
        First element is the uncorrected correlation coefficient, second element is the 
        lower bound of the CI and the third element is the upper bound of the CI.
    list
        First element is the Spearman-Brown corrected correlation coefficient, second element
        is the lower bound of the CI and the third element is the upper bound of the CI.
    list
        Difference as percentage of total trials of given type (TARGET and LURE respectively)
        between the two halves.
    """
    
    data_even, data_odd, data = assign_even_odd(data, id_var)
    even_target, odd_target, even_lure, odd_lure = len(data_even[data_even['StimType']=='TARGET']), len(data_odd[data_odd['StimType']=='TARGET']), len(data_even[data_even['StimType']=='LURE']), len(data_odd[data_odd['StimType']=='LURE'])
    target_diff, lure_diff = abs(even_target - odd_target) / len(data[data['StimType']=='TARGET']) *100, abs(even_lure - odd_lure) / len(data[data['StimType']=='TARGET']) * 100

    discrimination_even = discrimination_calculator(data_even)
    discrimination_odd = discrimination_calculator(data_odd)

    if nonparametric:
        r_raw = stats.spearmanr(discrimination_even['d_prime'], discrimination_odd['d_prime']).correlation
    else:
        r_raw = np.corrcoef(discrimination_even['d_prime'], discrimination_odd['d_prime'])[0][1]

    r_corrected = (2*r_raw) / (1+r_raw)  # Spearman-Brown correction
    result_raw = correl_ci(r_raw, 95, data_even)
    result_corrected = correl_ci(r_corrected, 95, data_even)
    return result_raw, result_corrected, [target_diff, lure_diff]


In [32]:
### Looping through each sample and condition ###
trial_types = ['OBJ', 'LOC']
result_even_odd = pd.DataFrame(columns=['data', 'type', 'splithalf', 'split_half_CI_low', 'split_half_CI_high', 'spearman_brown', 'spearman_brown_CI_low', 'spearman_brown_CI_high', 'target_diff (%)', 'lure_diff (%)'])
with pd.option_context('mode.chained_assignment', None):
    for trial_type in trial_types:
        print('='*25, trial_type, '='*25)
        for data, data_name in zip(data_list, data_names):
            print(data_name)
            data_dropped = data[['online ID', 'rec_trial_key.rt', 'TrialType', 'StimType', 'ResponseNew']].dropna()
            data_temp = data_dropped[(data_dropped['StimType']=='TARGET')|(data_dropped['StimType']=='LURE')]
            even_odd_result, corrected_result, difference = splithalf(data_temp[data_temp['TrialType']==trial_type], nonparametric=False)
            result_even_odd.loc[len(result_even_odd.index)] = [data_name, trial_type, even_odd_result[0], even_odd_result[1], even_odd_result[2], corrected_result[0], corrected_result[1], corrected_result[2], difference[0], difference[1]]
            
pd.options.display.float_format = '{:.6f}'.format
display(result_even_odd)


========================= OBJ =========================
------------------------- data_mr_young -------------------------
No filtering
Using regressors:  
------------------------- data_online_young -------------------------
No filtering
Using regressors:  
------------------------- data_online_old -------------------------
No filtering
Using regressors:  
------------------------- data_online -------------------------
No filtering
Using regressors:  
========================= LOC =========================
------------------------- data_mr_young -------------------------
No filtering
Using regressors:  
------------------------- data_online_young -------------------------
No filtering
Using regressors:  
------------------------- data_online_old -------------------------
No filtering
Using regressors:  
------------------------- data_online -------------------------
No filtering
Using regressors:  


,data,type,filter,regressor,splithalf,split_half_CI_low,split_half_CI_high,spearman_brown,spearman_brown_CI_low,spearman_brown_CI_high,target_diff (%),lure_diff (%)
0,data_mr_young,OBJ,none,,0.315520,0.258554,0.370298,0.479688,0.430471,0.526062,0.298507,0.497512
1,data_online_young,OBJ,none,,0.094855,0.036217,0.152843,0.173275,0.115613,0.229773,0.268097,0.804290
2,data_online_old,OBJ,none,,0.015970,-0.028659,0.060535,0.031437,-0.013189,0.075939,0.465116,0.568475
3,data_online,OBJ,none,,0.091540,0.056178,0.126673,0.167727,0.132975,0.202067,0.392927,0.654879
4,data_mr_young,LOC,none,,0.117001,0.055190,0.177918,0.209491,0.149224,0.268208,0.000000,0.903614
5,data_online_young,LOC,none,,0.186860,0.129573,0.242905,0.314882,0.260998,0.366810,0.446030,0.624442
6,data_online_old,LOC,none,,-0.052066,-0.096774,-0.007147,-0.109851,-0.154026,-0.065238,0.103093,1.907216
7,data_online,LOC,none,,0.049806,0.014151,0.085335,0.094886,0.059407,0.130126,0.228683,1.437439


# Bootstrapping

In [ ]:
def resampled_splithalf(data, id_var='online ID', nonparametric=False, level='participant', type='monte_carlo', i=None):
    """
    Draw two samples with replacement from the data and calculate the reliabiltiy as the correlation between the 
    participant effect sizes in the two samples.

    Parameters
    ----------
    data: dataframe
    id_var: str
        Name of the column holding participant names
    nonparametric: bool
        Calculate Spearman's rank-order correlation if True, Pearson's if False
    level: {'participant', 'trial'}
        Level of resampling. If 'participant', resample participants but not trials, if 'trial', resample trials but not participants.
    type: {'monte_carlo', 'permutation'}
        If 'monte_carlo', sample with replacement with frac=1.0, if 'premutation' sample without replacement with farc=0.5 and use Spearman-Brown correction.
    i: int
        Number of iteration.

    Returns
    -------
    float
        The calculated reliabiltiy as a correlation coefficient
    dataframe
        The participant effect sizes from the two samples
    """
    if type == 'monte_carlo':
        resample_frac = 1.0
        replacement = True
    elif type == 'permutation':
        resample_frac = 0.5
        replacement = False

    if level == 'trial':
        # Resampling by participant and stimulus type
        data_1, data_2 = pd.DataFrame(columns=data.columns), pd.DataFrame(columns=data.columns)
        for participant in data[id_var].unique():
            data_participant = data[data[id_var] == participant]
            if type == 'monte_carlo':
                data_1_target = data_participant[data_participant['StimType']=='TARGET'].sample(frac=resample_frac, replace=replacement)
                data_1_lure = data_participant[data_participant['StimType']=='LURE'].sample(frac=resample_frac, replace=replacement)
                data_2_target = data_participant[data_participant['StimType']=='TARGET'].sample(frac=resample_frac, replace=replacement)
                data_2_lure = data_participant[data_participant['StimType']=='LURE'].sample(frac=resample_frac, replace=replacement)
            elif type == 'permutation':
                data_1_target = data_participant[data_participant['StimType']=='TARGET'].sample(frac=resample_frac, replace=replacement)
                data_1_lure = data_participant[data_participant['StimType']=='LURE'].sample(frac=resample_frac, replace=replacement)
                data_2_target = data_participant[data_participant['StimType']=='TARGET'].drop(data_1_target.index)
                data_2_lure = data_participant[data_participant['StimType']=='LURE'].drop(data_1_lure.index)
            data_1 = pd.concat([data_1, data_1_target, data_1_lure])
            data_2 = pd.concat([data_2, data_2_target, data_2_lure])
        discrimination_1 = discrimination_calculator(data_1)
        discrimination_2 = discrimination_calculator(data_2)
    
    elif level == 'participant':
        data_1, data_2, data = assign_even_odd(data, id_var)
        discrimination_even = discrimination_calculator(data_1)
        discrimination_odd = discrimination_calculator(data_2)
        discrimination_1 = discrimination_even.sample(frac=resample_frac, replace=replacement)
        discrimination_2 = pd.DataFrame()
        for name in discrimination_1[id_var]:
            discrimination_2 = pd.concat([discrimination_2, discrimination_odd[discrimination_odd[id_var]==name]])
        
    
    # Gathering participant effect sizes
    discrimination_1.loc[:,'iteration'] = i
    discrimination_2.loc[:,'iteration'] = i
    discrimination_1.loc[:,'e_o'] = 1
    discrimination_2.loc[:,'e_o'] = 2
    discrimination = pd.concat([discrimination_1, discrimination_2])

    # Calculating split-half reliability with Pearson or Spearman
    if nonparametric:
        reliability = stats.spearmanr(discrimination_1['d_prime'], discrimination_2['d_prime']).correlation
    else:
        reliability = np.corrcoef(discrimination_1['d_prime'], discrimination_2['d_prime'])[0][1]

    if type == 'permutation':
        reliability = (2*reliability) / (1+reliability)  # Spearman-Brown correction

    return reliability, discrimination


def bootstrap_loop(n, data, level='participant', type = 'monte_carlo'):
    """
    Loop resampled_splithalf() n times to perform bootstrapping.

    Parameters
    ----------
    n: int
        Number of resamplings to perform
    data: dataframe
        Dataframe to analyse
    level: {'participant', 'trial'}
        Level of resampling. If 'participant', resample participants but not trials, if 'trial', resample trials but not participants.
    type: {'monte_carlo', 'permutation'}
        If 'monte_carlo', sample with replacement with frac=1.0, if 'premutation' sample without replacement with farc=0.5 and use Spearman-Brown correction.

    Returns
    -------
    list
        The correlation coefficients from each resampling
    dataframe
        The participant effect sizes from all resamplings
    """
    reliability_distribution = []
    effect_sizes = pd.DataFrame(columns=['online ID', 'd_prime', 'p_old_target', 'p_old_lure', 'iteration', 'e_o'])
    for i in range(n):
        print(i, end=" ")
        reliability, discrimination = resampled_splithalf(data, id_var='online ID', nonparametric=False, level=level, type=type, i=i)
        reliability_distribution.append(reliability)
        effect_sizes = pd.concat([effect_sizes, discrimination])
    return reliability_distribution, effect_sizes


def bootstrapping_analyses(n, level='participant', type = 'monte_carlo', data_list=data_list, data_names=data_names):
    """
    Run boostrap_loop() on all samples and conditions. Gather results into dataframes.

    Parameters
    ----------
    n: int
        Number of resamplings to perform
    level: {'participant', 'trial'}
        Level of resampling. If 'participant', resample participants but not trials, if 'trial', resample trials but not participants.
    type: {'monte_carlo', 'permutation'}
        If 'monte_carlo', sample with replacement with frac=1.0, if 'premutation' sample without replacement with farc=0.5 and use Spearman-Brown correction.
    data_list: list of dataframes
        List of dataframes to analyize.
    data_names: list of str
        Lists of the names of the dataframes analyized.

    Returns
    -------
    dataframe
        Participant effect sizes labeled by data_set and condition
    dataframe
        Reliability results as correlation coefficients labeled by data_set and condition
    """
    trial_types = ['OBJ', 'LOC']
    reliability_distribution = pd.DataFrame(columns=['r', 'data_set', 'trial_type'])
    effect_size_distribution = pd.DataFrame(columns=['online ID', 'd_prime', 'iteration', 'e_o', 'data_set', 'trial_type'])
    with pd.option_context('mode.chained_assignment', None):
        for trial_type in trial_types:
            for df, data_name in zip(data_list, data_names):
                data = df[df['TrialType']==trial_type]
                print('\n', 'Resampling: ', trial_type, data_name)
                reliability_list, effect_sizes = bootstrap_loop(n, data, level=level, type=type)
                reliability_df = pd.DataFrame(reliability_list, columns=['r'])
                reliability_df['data_set'] = data_name
                reliability_df['trial_type'] = trial_type
                effect_sizes['data_set'] = data_name
                effect_sizes['trial_type'] = trial_type

                effect_size_distribution = pd.concat([effect_size_distribution, effect_sizes[['online ID', 'd_prime','iteration', 'e_o', 'data_set', 'trial_type']]])
                reliability_distribution = pd.concat([reliability_distribution, reliability_df])
    # display(effect_size_distribution,reliability_distribution)
    return effect_size_distribution, reliability_distribution

# Run bootstrapping
effect_size_distribution, reliability_distribution = bootstrapping_analyses(n=10, level='participant', type='permutation')


# Summarize results

In [58]:
def effect_size_agg(data):
    """
    Calculate mean participant effect sizes and percentile bootsrapping 95% CIs for each participant in each dataset and condition.

    Parameters
    ----------
    data: dataframe
        Dataframe including all effect sizes for all participants calculated in all resamples. Needs 'online ID', 'data_set and 'trial_type' columns 
        denoting participant names, the dataset used for calculation and the condition, respectively.

    Returns
    -------
    dataframe
        Dataframe containing mean effect size and its 95% bootstrapped CI lower and upper bounds as values and as differences from the mean, 
        as well as the data_set and trial_type.
    """
    effect_size_summary = pd.DataFrame(columns=['online ID', 'd_mean', 'd_lower','d_higher', 'd_lower_diff', 'd_higher_diff', 'data_set', 'trial_type'])
    for data_name in data['data_set'].unique():
        for trial_type in data['trial_type'].unique():
            data_temp = data[(data['data_set']==data_name)&(data['trial_type']==trial_type)]
            summary_temp = pd.DataFrame()
            summary_temp['online ID'] = data_temp['online ID'].unique()
            summary_temp['ID_num'] = [j for i, j in zip(summary_temp['online ID'], range(len(summary_temp['online ID'])))]

            summary_temp['d_mean'] = [np.mean(data_temp[data_temp['online ID']==name]['d_prime']) for name in data_temp['online ID'].unique()]
            summary_temp['d_lower'] = [np.percentile(data_temp[data_temp['online ID']==name]['d_prime'],2.5) for name in data_temp['online ID'].unique()]
            summary_temp['d_higher'] = [np.percentile(data_temp[data_temp['online ID']==name]['d_prime'],97.5) for name in data_temp['online ID'].unique()]

            summary_temp['d_lower_diff'] = summary_temp['d_mean'] - summary_temp['d_lower']
            summary_temp['d_higher_diff'] = summary_temp['d_higher'] - summary_temp['d_mean']
            summary_temp['data_set'] = data_name
            summary_temp['trial_type'] = trial_type
            effect_size_summary = pd.concat([effect_size_summary, summary_temp])

    return effect_size_summary

def bs_reliability_agg(data):
    """
    Calculate mean reliability, its percentile bootstrapping 95% CI for each dataset and condition used in the analysis.

    Parameters
    ----------
    data: dataframe
        Needs 'r', 'data_set and 'trial_type' columns denoting measured reliability, the dataset used for calculation and 
        the condition, respectively.

    Returns
    -------
    dataframe
        Dataframe containing mean reliability and its 95% bootstrapped CI lower and upper bounds as values and as differences from the mean, 
        as well as the data_set and trial_type.
    """
    reliability_summary = pd.DataFrame(columns=['r_mean', 'r_lower','r_higher', 'r_lower_diff', 'r_higher_diff', 'data_set', 'trial_type'])
    for data_name in data['data_set'].unique():
        for trial_type in data['trial_type'].unique():
            data_temp = data[(data['data_set']==data_name)&(data['trial_type']==trial_type)]
            summary_temp = pd.DataFrame()

            summary_temp['r_mean'] = [np.mean(data_temp['r'])]
            summary_temp['r_lower'] = [np.percentile(data_temp['r'],2.5)]
            summary_temp['r_higher'] = [np.percentile(data_temp['r'],97.5)]

            summary_temp['r_lower_diff'] = summary_temp['r_mean'] - summary_temp['r_lower']
            summary_temp['r_higher_diff'] = summary_temp['r_higher'] - summary_temp['r_mean']
            summary_temp['data_set'] = data_name
            summary_temp['trial_type'] = trial_type
            reliability_summary = pd.concat([reliability_summary, summary_temp])

    return reliability_summary

# Run summary functions
effect_size_summary = effect_size_agg(effect_size_distribution)
reliability_summary = bs_reliability_agg(reliability_distribution)
effect_size_summary_1 = effect_size_agg(effect_size_distribution[effect_size_distribution['e_o']==1])
effect_size_summary_2 = effect_size_agg(effect_size_distribution[effect_size_distribution['e_o']==2])
# display(effect_size_summary.head(), effect_size_summary_1.head(), effect_size_summary_2.head(), reliability_summary)


# Participan effect sizes and CIs

In [394]:
# Effect sizes
effect_sizes_all = pd.read_csv(r"")
display(effect_sizes_all['data_set'].unique())

# % containing 0
effect_sizes = effect_sizes_all[effect_sizes_all['data_set'] != 'data_online']
effect_sizes_0 = effect_sizes[effect_sizes['d_lower']<=0]
print("Percent of participant CIs containing 0: ", len(effect_sizes_0)/len(effect_sizes)*100, "%")
print("In different samples:")
for data_set in effect_sizes['data_set'].unique():
    df_temp = effect_sizes[effect_sizes['data_set']==data_set]
    df_temp_0 = df_temp[df_temp['d_lower']<=0]
    print("In", data_set, ": ", len(df_temp_0)/len(df_temp)*100, "%")
print("In different task versions:")
for trial_type in effect_sizes['trial_type'].unique():
    df_temp = effect_sizes[effect_sizes['trial_type']==trial_type]
    df_temp_0 = df_temp[df_temp['d_lower']<=0]
    print("In", trial_type, ": ", len(df_temp_0)/len(df_temp)*100, "%")

# Overlap
def percent_overlap(name, data):
    ci = np.array(data[data['online ID']==name][['d_lower', 'd_higher']])[0]
    
    rest = data[data['online ID']!=name][['d_mean', 'd_lower', 'd_higher']]
    rest['CI_overlap'] = np.where((rest['d_lower']>ci[1])|(rest['d_higher']<ci[0]),0,1)
    rest['point_overlap'] = np.where((rest['d_mean']>ci[0])&(rest['d_mean']<ci[1]), 1, 0)

    return(np.sum(rest['CI_overlap'])/len(rest)*100, np.sum(rest['point_overlap'])/len(rest)*100)

overlap_results = pd.DataFrame(columns=['trial_type', 'data_frame', 'CI overlap (%)', 'point estimate overlap (%)'])
for trial_type in effect_sizes_all['trial_type'].unique():
    df_trial_type = effect_sizes_all[effect_sizes_all['trial_type']==trial_type]
    for data_set in effect_sizes_all['data_set'].unique():
        df_temp = df_trial_type[df_trial_type['data_set']==data_set]
        ci_overlap_list = []
        point_overlap_list = []
        for name in df_temp['online ID']:
            ci_overlap, point_overlap = percent_overlap(name, df_temp)
            ci_overlap_list.append(ci_overlap)
            point_overlap_list.append(point_overlap)
        overlap_results.loc[len(overlap_results.index)] = [trial_type, data_set, np.mean(ci_overlap_list), np.mean(point_overlap_list)]

display('Mean Ci overlap', np.mean(overlap_results[overlap_results['data_frame']!='data_online']['CI overlap (%)']), 
'Mean containing point estimate', np.mean(overlap_results[overlap_results['data_frame']!='data_online']['point estimate overlap (%)']))

labels_dataset = {'data_mr_young':'MRI data (young)', 'data_online_young':'Online data (young)', 
                  'data_online_old':'Online data (old)', 'data_online':'Online data (all)'}
version_dict = {'OBJ':'Object version', 'LOC':'Location version'}

overlap_results['data_frame'] = [labels_dataset[name] for name in overlap_results['data_frame']]
overlap_results['trial_type'] = [version_dict[name] for name in overlap_results['trial_type']]
overlap_results.columns = ['Task version', 'Sample', 'CI overlap (average %)', 'Point estimate overlap (average %)']
display(overlap_results.round(2))

array(['data_mr_young', 'data_online_young', 'data_online_old',
       'data_online'], dtype=object)

Percent of participant CIs containing 0:  89.47368421052632 %
In different samples:
In data_mr_young :  84.52380952380952 %
In data_online_young :  81.91489361702128 %
In data_online_old :  96.34146341463415 %
In different task versions:
In OBJ :  90.05847953216374 %
In LOC :  88.88888888888889 %


,trial_type,data_frame,CI overlap (%),point estimate overlap (%)
0,OBJ,data_mr_young,98.838560,79.790941
1,OBJ,data_online_young,99.537465,83.811286
2,OBJ,data_online_old,99.789220,82.821439
3,OBJ,data_online,99.370155,81.474079
4,LOC,data_mr_young,97.328688,70.615563
5,LOC,data_online_young,98.427382,77.983349
6,LOC,data_online_old,97.711533,81.135200
7,LOC,data_online,97.771318,78.258236


'Mean Ci overlap'

98.60547458926958

'Mean containing point estimate'

79.35962970483853

,Task version,Sample,CI overlap (average %),Point estimate overlap (average %)
0,Object version,MRI data (young),98.84,79.79
1,Object version,Online data (young),99.54,83.81
2,Object version,Online data (old),99.79,82.82
3,Object version,Online data (all),99.37,81.47
4,Location version,MRI data (young),97.33,70.62
5,Location version,Online data (young),98.43,77.98
6,Location version,Online data (old),97.71,81.14
7,Location version,Online data (all),97.77,78.26
